# 📚 打造 Stable Diffusion 的 WebUI

### 1. 安裝必要套件

In [1]:
!pip install diffusers transformers accelerate safetensors huggingface_hub gradio --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler, DPMSolverMultistepScheduler, HeunDiscreteScheduler  # 穩定擴散管道和調度器
import matplotlib.pyplot as plt  # 繪圖套件
import torch            # 深度學習套件
import gc              # 垃圾回收套件
import gradio as gr        # 網頁介面套件
import random            # 隨機數生成套件

### 2. 指定並讀入多個模型、排程器、隨機提詞讓使用者可以依喜好自行選擇

In [3]:
# 定義 model 選項
model_options = {
    "sd-v1-5": "stable-diffusion-v1-5/stable-diffusion-v1-5",
    "sd-2-1": "stabilityai/stable-diffusion-2-1",
    "sd-turbo": "stabilityai/sd-turbo"
}

In [4]:
# 定義 scheduler 選項
scheduler_options = {
    "UniPCMultistep": UniPCMultistepScheduler,
    "DPM++ 2M Karras": DPMSolverMultistepScheduler,
    "Heun": HeunDiscreteScheduler
}

In [5]:
# 供使用者隨機使用的 prompts 選項
random_prompts = [
    ("A photorealistic ocean with a giant transparent piano floating in the sky, playing glowing musical notes that fall like rain, ultra high quality, cinematic lighting",
     "realistic glass textures, glowing notes, reflective ocean waves"),
    ("A photorealistic lighthouse made of glowing feathers in a stormy neon sky, shining a rainbow beam, ultra high quality, cinematic lighting",
     "realistic feather details, vibrant neon colors, dramatic storm effects"),
    ("A photorealistic giant clock made of blooming roses floating in a golden sunset sky, with petals falling gently, ultra high quality, cinematic lighting",
     "realistic rose textures, warm golden light, soft petal effects"),
    ("A photorealistic tree with sparkling candy fruits glowing like stars in a midnight galaxy sky, ultra high quality, cinematic lighting",
     "realistic candy textures, glowing star effects, deep galaxy colors"),
    ("A photorealistic waterfall made of shimmering crystal butterflies flowing down a cliff in a misty forest, ultra high quality, cinematic lighting",
     "realistic crystal reflections, soft mist effects, detailed butterfly wings"),
    ("A photorealistic tornado made of exploding diamonds swirling over a molten gold desert, with glittering shards raining down, ultra high quality, cinematic lighting.",
     "realistic diamond sparkle, molten gold reflections, dramatic explosion effects."),
    ("A photorealistic phoenix made of shimmering crystals rising from a sea of flaming sapphire, with radiant wings igniting the sky, ultra high quality, cinematic lighting.",
     "realistic crystal feathers, vibrant sapphire flames, glowing rebirth effects."),
    ("A photorealistic giant whale floating in a sky of swirling rainbows, exhaling a cascade of sparkling stardust, ultra high quality, cinematic lighting.",
     "realistic whale skin, vibrant rainbow swirls, sparkling stardust trails."),
    ("A photorealistic giant spider web woven with glowing starlight, spanning a crimson nebula sky, ultra high quality, cinematic lighting.",
     "realistic web threads, radiant starlight glow, deep crimson nebula."),
    ("A photorealistic icy chariot charging through an aurora battlefield, with frozen lightning bolts shattering the sky, ultra high quality, cinematic lighting.",
     "realistic ice textures, vibrant aurora colors, dramatic lightning effects."),
    ("A photorealistic seahorse made of translucent glass swimming through a galaxy vortex, with stars swirling in its wake, ultra high quality, cinematic lighting.",
     "realistic glass scales, swirling galaxy patterns, star trail effects.")
]

# 定義生成函數
job = None  # 用於儲存生成任務以便中斷

### 3. 定義要用來生成的函式

In [6]:
# 定義生成函數
job = None  # 用於儲存生成任務以便中斷

def generate_images(prompt, use_enhance, enhance_text, use_negative, negative_text,
                   use_custom_seed, custom_seed, height, width, steps, num_images,
                   cfg_scale, eta, selected_models, selected_scheduler):
    global job
    job = "running"

    # 圖片的長寬
    height = int(height)
    width = int(width)
    if height % 8 != 0 or width % 8 != 0:
        raise ValueError("高度和寬度必須是8的倍數！")

    # 產生隨機種子設定
    if use_custom_seed:
        base_seed = int(custom_seed)
    else:
        base_seed = random.randint(0, 2**32 - 1)
    seeds = [base_seed + i for i in range(num_images)]

    final_prompt = prompt
    if use_enhance and enhance_text:
        final_prompt = prompt + ", " + enhance_text
    final_negative = negative_text if use_negative else None

    # 限制 sd-turbo 步數最高 40
    steps = min(steps, 40)

    # 初始化每個模型的圖片列表
    model_outputs = {"sd-v1-5": [], "sd-2-1": [], "sd-turbo": []}

    for model_name in selected_models:
        if job != "running":
            break
        try:
            pipe = StableDiffusionPipeline.from_pretrained(
                model_options[model_name],
                torch_dtype=torch.float16,
                use_safetensors=True
            ).to("cuda")

            # 調整排程器設定
            scheduler_class = scheduler_options[selected_scheduler]
            if selected_scheduler == "DPM++ 2M Karras":
                pipe.scheduler = scheduler_class.from_config(pipe.scheduler.config, use_karras_sigmas=True)
            else:
                pipe.scheduler = scheduler_class.from_config(pipe.scheduler.config)

            images = []
            for i in range(num_images):
                if job != "running":
                    break
                generator = torch.Generator("cuda").manual_seed(seeds[i])
                with torch.no_grad():
                    image = pipe(
                        prompt=final_prompt,
                        negative_prompt=final_negative,
                        height=height,
                        width=width,
                        num_inference_steps=steps,
                        guidance_scale=cfg_scale,
                        eta=eta,
                        generator=generator
                    ).images[0]
                    images.append(image)
                model_outputs[model_name] = images
                # 動態更新輸出
                yield model_outputs["sd-v1-5"], model_outputs["sd-2-1"], model_outputs["sd-turbo"], f"使用的 random seeds: {seeds}"

            del pipe
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"模型 {model_name} 生成失敗: {str(e)}")
            model_outputs[model_name] = []
            yield model_outputs["sd-v1-5"], model_outputs["sd-2-1"], model_outputs["sd-turbo"], f"使用的 random seeds: {seeds} (錯誤: {str(e)})"

    job = None
    return model_outputs["sd-v1-5"], model_outputs["sd-2-1"], model_outputs["sd-turbo"], f"使用的 random seeds: {seeds}"

# 定義用來隨機生 Prompt 的函數
def random_prompt():
    prompt, enhance = random.choice(random_prompts)
    return prompt, enhance, f"隨機選擇的描述：{prompt}"

# 定義暫停生成的函數
def stop_generation():
    global job
    job = "stopping"
    return "已暫停生成！"

### 4. 打造 Gradio Web App

In [7]:
# 預設的prompt內容
default_enhance = "realistic textures, vibrant colors, cinematic effects"
default_negative = "blurry, low quality, distorted shapes"


# CSS設定
with gr.Blocks(css="""
    .gradio-container {
        background-color: #F5F7FA;
        padding: 20px;
    }
    .gr-button {
        font-size: 18px;
        background: linear-gradient(to right, #4CAF50, #2196F3);
        color: white;
        border: none;
        border-radius: 5px;
    }
    .gr-box {
        background-color: #FFFFFF;
        border-radius: 10px;
        box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
    }
""") as demo:
    # 標題與說明
    gr.Markdown("""
    # 🎨 奇幻畫境生成器
    探索超現實世界！輸入描述或隨機生成，創造你的奇幻畫作！
    """)

    # UI介面中的內容設定
    with gr.Row():
        # 左邊column設定
        with gr.Column(scale=6):
            # 輸入prompt
            prompt = gr.Textbox(
                label="描述你的畫面",
                placeholder="例如：漂浮在星空中的巨大玻璃鋼琴",
                lines=3,
                value="A photorealistic ocean with a giant transparent piano floating in the sky, playing glowing musical notes that fall like rain, ultra high quality, cinematic lighting"
            )
            random_btn = gr.Button("🎉 隨機生成驚喜！～")
            random_info = gr.Label(label="隨機描述結果")
            # 輸入加強prompt
            with gr.Row():
                use_enhance = gr.Checkbox(label="加強描述", value=True)
                enhance_text = gr.Textbox(label="加強內容", value=default_enhance)
            # 輸入負面prompt
            with gr.Row():
                use_negative = gr.Checkbox(label="排除不需要的元素", value=True)
                negative_text = gr.Textbox(label="排除內容", value=default_negative)
            # 輸入Random Seed
            with gr.Row():
                use_custom_seed = gr.Checkbox(label="自訂 Random Seed", value=False)
                custom_seed = gr.Number(label="指定 Seed", value=666)
            # 輸入圖片長寬設定
            with gr.Row():
                height = gr.Dropdown(["768", "1024", "2048"], label="高度", value="2048")
                width = gr.Dropdown(["768", "1024", "2048"], label="寬度", value="2048")
            # 更多超參數調整
            with gr.Row():
                steps = gr.Slider(1, 100, value=25, step=1, label="生成步數 (Steps)")
                cfg_scale = gr.Slider(1, 20, value=7.5, step=0.5, label="CFG Scale")
                eta = gr.Slider(0, 1, value=0.2, step=0.1, label="Eta (隨機性)")
                num_images = gr.Slider(1, 10, value=2, step=1, label="生成張數")
            # 相關參數解釋說明
            gr.Markdown("""
            ### 參數說明：
            - **生成步數 (Steps)**：圖片生成次數，越多越細緻但越慢，預設 25 步。
            - **CFG Scale**：圖片跟描述的貼合度，1-20，數字越大越逼近描述，預設 7.5。
            - **Eta (隨機性)**：圖片的隨機變化，0-1，數字越小越穩定，預設 0.2。
            - **生成張數**：一次生成幾張圖，1-10 張，預設 2 張以免太慢。
            """)
            # 輸入模型跟排程器調整
            with gr.Row():
                model_select = gr.CheckboxGroup(choices=["sd-v1-5", "sd-2-1", "sd-turbo"], label="選擇模型", value=["sd-turbo"])
                scheduler_select = gr.Dropdown(choices=["UniPCMultistep", "DPM++ 2M Karras", "Heun"], label="選擇 Scheduler", value="DPM++ 2M Karras")
            # 相關模型跟排程器解釋說明
            gr.Markdown("""
            ### 模型選擇說明：
            - **sd-v1-5**：細節豐富，適合逼真畫風，但較慢。
            - **sd-2-1**：構圖更好，適合複雜場景，速度中等。
            - **sd-turbo**：生成快速，適合快速預覽，但細節較少。
            **建議**：想快速看效果用 `sd-turbo`，想要高品質用 `sd-2-1`。

            ### Scheduler 選擇說明：
            - **UniPCMultistep**：穩定，細節好，但較慢。
            - **DPM++ 2M Karras**：速度快，細節佳，推薦使用。
            - **Heun**：簡單場景可用，細節普通，速度中等。
            **建議**：大多數情況選 `DPM++ 2M Karras`，速度與品質兼顧。
            """)
            # 開始 & 暫停生成
            with gr.Row():
                generate_btn = gr.Button("🚀 開始生成！")
                stop_btn = gr.Button("⏹ 暫停生成")

        # 右邊column設定
        with gr.Column(scale=6):
            gallery_v15 = gr.Gallery(label="sd-v1-5 結果", columns=3, object_fit="contain", height="1200px")
            gallery_v21 = gr.Gallery(label="sd-2-1 結果", columns=3, object_fit="contain", height="1200px")
            gallery_turbo = gr.Gallery(label="sd-turbo 結果", columns=3, object_fit="contain", height="1200px")
            seed_info = gr.Label(label="使用的 Random Seeds")

    # 隨機prompt按鈕
    random_btn.click(
        fn=random_prompt,
        inputs=[],
        outputs=[prompt, enhance_text, random_info]
    )
    # 開始生成按鈕
    generate_btn.click(
        fn=generate_images,
        inputs=[prompt, use_enhance, enhance_text, use_negative, negative_text,
                use_custom_seed, custom_seed, height, width, steps, num_images,
                cfg_scale, eta, model_select, scheduler_select],
        outputs=[gallery_v15, gallery_v21, gallery_turbo, seed_info]
    )
    # 停止生成按鈕
    stop_btn.click(
        fn=stop_generation,
        inputs=[],
        outputs=[seed_info]
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3e4f9bddb80b30abbe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3e4f9bddb80b30abbe.gradio.live
